In [1]:
%%bash
docker run \
  --rm \
  --name omnisci_test \
  --ipc=host \
  -d --runtime=nvidia \
  -v $(readlink -f ..)/data:/data \
  -v $(readlink -f ..)/scripts:/src \
  -p 6273-6280:6273-6280 \
  omnisci/core-os-cuda:v5.2.1

7abc62862fad31d9f9f750806070723850e844fa4eb9e4cde8c642ef0e64028f


In [2]:
%%bash
docker exec omnisci_test /bin/bash -c \
    "cat /src/load_dataset.sql | /omnisci/bin/omnisql -p 'HyperInteractive' -t"

User admin connected to database omnisci
Result
Loaded: 21347420 recs, Rejected: 0 recs in 4.259000 secs
1 rows returned.
Execution time: 4329 ms, Total time: 4329 ms
User admin disconnected from database omnisci


In [3]:
%%bash
docker exec omnisci_test /bin/bash -c \
    "echo 'SELECT * FROM online_retail LIMIT 10;' | /omnisci/bin/omnisql -p 'HyperInteractive' -t"

User admin connected to database omnisci
Invoice|StockCode|Description|Quantity|InvoiceDate|Price|CustomerID|Country
490928|71053|WHITE METAL LANTERN|3|2009-12-08 13:41:00|3.75|14085.0|United Kingdom
490928|21875|KINGS CHOICE MUG|6|2009-12-08 13:41:00|1.25|14085.0|United Kingdom
490928|21067|VINTAGE RED TEATIME MUG|6|2009-12-08 13:41:00|1.25|14085.0|United Kingdom
490928|21622|VINTAGE UNION JACK CUSHION COVER|6|2009-12-08 13:41:00|4.95|14085.0|United Kingdom
491662|84406B|CREAM CUPID HEARTS COAT HANGER|1|2009-12-13 11:16:00|3.25|15169.0|United Kingdom
491665|82552|WASHROOM METAL SIGN|3|2009-12-13 11:32:00|1.25|14685.0|United Kingdom
491043|84029E|RED WOOLLY HOTTIE WHITE HEART.|2|2009-12-09 09:51:00|7.78|NULL|United Kingdom
490928|21527|RETRO SPOT TRADITIONAL TEAPOT|1|2009-12-08 13:41:00|7.95|14085.0|United Kingdom
490745|21070|VINTAGE BILLBOARD MUG|1|2009-12-07 18:02:00|2.57|NULL|United Kingdom
490353|22042|CHRISTMAS CARD SINGING ANGEL|12|2009-12-04 16:37:00|0.42|15514.0|United Kingdom

In [1]:
from pymapd import connect
from time import time
import pandas as pd
import numpy as np
import json

In [2]:
with open('test_result.json') as json_file:
    test_result = json.load(json_file)

In [3]:
con = connect(user="admin", password="HyperInteractive", host="localhost", dbname="omnisci")

In [4]:
test_result["timings"]["omnisci"] = {}

for label, query in test_result["queries"].items():
    print(f"Executing {label} -> {query}")
    test_result["timings"]["omnisci"][label] = []
    for i in range(0, 30):
        try:
            start = time()
            pd.read_sql_query(query, con=con)  # <-un po' più veloce ma usa ipc (arrow)
            # df = con.select_ipc_gpu(query)  # <-analogo
            # result = list(con.execute(query))  #NO IPC
            end = time()

            total = end-start
            test_result["timings"]["omnisci"][label].append(total)
        except:
            test_result["timings"]["omnisci"][label].append(0)
            print(f"Query {query} not executed.")
            break

with open('test_result.json', 'w') as outfile:
    json.dump(test_result, outfile, indent=2)

Executing Q1 -> SELECT Invoice, SUM(Price*Quantity) as invoice_sum FROM online_retail GROUP BY Invoice
Executing Q2 -> SELECT CustomerID, SUM(Price*Quantity) as customer_sum FROM online_retail GROUP BY CustomerID
Executing Q3 -> SELECT Country, SUM(Price*Quantity) as country_sum FROM online_retail GROUP BY Country
Executing Q4 -> SELECT StockCode, COUNT(*) as n_items FROM online_retail GROUP BY StockCode ORDER BY n_items DESC
Executing Q5 -> SELECT Invoice, SUM(Price*Quantity) as invoice_sum FROM online_retail GROUP BY Invoice ORDER BY invoice_sum DESC
Executing Q6 -> SELECT InvoiceDate, AVG(Price*Quantity) as avg_total FROM online_retail GROUP BY InvoiceDate ORDER BY avg_total
Executing Q7 -> SELECT Invoice, SUM(Price*Quantity) as invoice_sum FROM online_retail WHERE Country='France' GROUP BY Invoice ORDER BY invoice_sum DESC
Executing Q8 -> SELECT CustomerID, Price*Quantity as total_price FROM online_retail WHERE Quantity > 20 ORDER BY total_price DESC
Query SELECT CustomerID, Price*

## Join tests

In [5]:
# query = "SELECT Invoice, stock_codes.Price FROM online_retail JOIN stock_codes ON online_retail.StockCode = stock_codes.StockCode"

# start = time()
# pd.read_sql_query(query, con=con)
# end = time()

# total = end-start
# print(total)

In [6]:
# query = "SELECT Invoice, Price FROM online_retail"

# start = time()
# pd.read_sql_query(query, con=con)
# end = time()

# total = end-start
# print(total)

In [7]:
%%bash
docker stop omnisci_test

omnisci_test
